In [81]:
import boto3
import pandas as pd
from io import StringIO
import time

In [82]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [83]:
AWS_ACCESS_KEY = config.get('AWS','KEY')
AWS_SECRET_KEY = config.get('AWS','SECRET')
AWS_REGION = "ap-south-1"
SCHEMA_NAME = "covid-19"
S3_STAGING_DIR = "s3://dexter1-covid-test-bucket/output/"
S3_BUCKET_NAME = "dexter1-covid-test-bucket"
S3_OUTPUT_DIRECTORY = "output"

In [84]:
athena_client = boto3.client("athena",
                            aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_KEY,
                            region_name=AWS_REGION)

In [85]:
Dict = {}
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            # This function loads only first 1000 rows
            results=client.get_query_execution(QueryExecutionId=query_response['QueryExecutionId'])
            if 'QueryExecution' in results and \
                'Status' in results['QueryExecution'] and \
                'State' in results['QueryExecution']['Status']:
                state = results['QueryExecution']['Status']['State']
                if state == 'SUCCEEDED':
                    client.get_query_results(
                        QueryExecutionId=query_response['QueryExecutionId']
                    )
                    break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

In [116]:
response = athena_client.start_query_execution(
    QueryString="select * from enigma_jhud",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

In [117]:
response

{'QueryExecutionId': '7ed0c0e9-5ee8-4c50-82b3-390c22151dc1',
 'ResponseMetadata': {'RequestId': 'fa71441c-8757-4596-bde8-85dff9913cf1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Jan 2023 12:27:12 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fa71441c-8757-4596-bde8-85dff9913cf1'},
  'RetryAttempts': 0}}

In [118]:
enigma_jhud = download_and_load_query_results(athena_client, response)

In [119]:
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [120]:
response = athena_client.start_query_execution(
    QueryString="select * from covid_19_testing_states_daily_states_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
covid_19_testing_states_daily=download_and_load_query_results(athena_client, response)

In [121]:
covid_19_testing_states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210220,UT,366034,NaN,1507875.0,NaN,totalTestsViral,2788882,260.0,14421.0,...,NaN,8.0,39.0,70f3e22ea3d10f99d5f3c09c55ba95fa1b8aaabb,0.0,0.0,0.0,0.0,0.0,NaN
1,20210220,VA,561812,117662.0,NaN,195.0,totalTestEncountersViral,5728208,1594.0,23436.0,...,NaN,99.0,67.0,75d813bab6075e36b3ed1d3bbbfe18f6692e3959,0.0,0.0,0.0,0.0,0.0,NaN
2,20210220,VI,2575,NaN,43564.0,108.0,posNeg,46139,NaN,NaN,...,NaN,0.0,0.0,7ca160663de572688bb23d17943b6f59863f5fd0,0.0,0.0,0.0,0.0,0.0,NaN
3,20210220,VT,14359,411.0,309335.0,NaN,totalTestsViral,1009285,39.0,NaN,...,NaN,3.0,0.0,5156647b94cb2e59c9e4e26be1943e4827a99f13,0.0,0.0,0.0,0.0,0.0,NaN
4,20210220,WA,332904,17485.0,NaN,NaN,totalTestEncountersViral,5048054,608.0,18969.0,...,NaN,19.0,35.0,8150e925fc2fb429eeb347109e52f7b99ba00f17,0.0,0.0,0.0,0.0,0.0,NaN


In [122]:
response = athena_client.start_query_execution(
    QueryString="select * from covid_19_testing_us_daily_us_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
covid_19_testing_us_daily=download_and_load_query_results(athena_client, response)

In [123]:
covid_19_testing_us_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [104]:
response = athena_client.start_query_execution(
    QueryString="select * from covid_19_testing_us_total_latest_us_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
covid_19_testing_us_total_latest=download_and_load_query_results(athena_client, response)

In [105]:
covid_19_testing_us_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [108]:
response = athena_client.start_query_execution(
    QueryString="select * from nytimes_data_in_usa_us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
nytimes_data_in_usa_us_county=download_and_load_query_results(athena_client, response)

In [109]:
response = athena_client.start_query_execution(
    QueryString="select * from nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
nytimes_data_in_usa_us_states=download_and_load_query_results(athena_client, response)

In [110]:
response = athena_client.start_query_execution(
    QueryString="select * from static_datasets_countrycodecountrycode",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
static_datasets_countrycode=download_and_load_query_results(athena_client, response)

In [111]:
response = athena_client.start_query_execution(
    QueryString="select * from static_datasets_countypopulationcountrypopulation",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
static_datasets_countypopulation=download_and_load_query_results(athena_client, response)

In [125]:
response = athena_client.start_query_execution(
    QueryString="select * from static_datasets_states_abvstate_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
static_datasets_states_abv=download_and_load_query_results(athena_client, response)

In [124]:
response = athena_client.start_query_execution(
    QueryString="select * from usa_hospital_beds_rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)
usa_hospital_beds=download_and_load_query_results(athena_client, response)

In [129]:
static_datasets_states_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [128]:
newHeader = static_datasets_states_abv.iloc[0]
static_datasets_states_abv = static_datasets_states_abv[1:]
static_datasets_states_abv.columns = newHeader

In [131]:
factCovid1 = enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid2 = covid_19_testing_states_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid = pd.merge(factCovid1,factCovid2, on="fips", how="inner")

In [133]:
factCovid.shape

(27992, 13)

In [134]:
dimRegion1 = enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
dimRegion2 = nytimes_data_in_usa_us_county[['fips','county','state']]
dimRegion = pd.merge(dimRegion1,dimRegion2, on="fips", how="inner")

In [180]:
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [136]:
dimHospital = usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]

In [182]:
dimHospital.head()

,fips,state_name,latitude,longtitude,hq_address,hospital_name,hospital_type,hq_city,hq_state
0,27093.0,Minnesota,45.119098,-94.528802,612 S Sibley Ave,Meeker County Memorial Hospital,Critical Access Hospital,Litchfield,MN
1,27095.0,Minnesota,46.073181,-93.662491,200 Elm St N,Mille Lacs Health System,Critical Access Hospital,Onamia,MN
2,27141.0,Minnesota,45.558437,-93.590102,911 Northland Dr,M Health Fairview Northland Medical Center,Short Term Acute Care Hospital,Princeton,MN
3,27097.0,Minnesota,45.967300,-94.362700,815 2nd St Se,CHI St Gabriel Health (FKA St Gabriels Hospital),Critical Access Hospital,Little Falls,MN
4,27101.0,Minnesota,43.992305,-95.759840,2042 Juniper Ave,Murray County Medical Center,Critical Access Hospital,Slayton,MN


In [137]:
dimDate = covid_19_testing_states_daily[['fips','date']]

In [181]:
dimDate.head()

,fips,date,year,month,day_of_week
0,49.0,2021-02-20,2021,2,5
1,51.0,2021-02-20,2021,2,5
2,78.0,2021-02-20,2021,2,5
3,50.0,2021-02-20,2021,2,5
4,53.0,2021-02-20,2021,2,5


In [139]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')

C:\Users\shubh\AppData\Local\Temp\ipykernel_976\572748324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [140]:
dimDate.head()

,fips,date
0,49.0,2021-02-20
1,51.0,2021-02-20
2,78.0,2021-02-20
3,50.0,2021-02-20
4,53.0,2021-02-20


In [141]:
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek

C:\Users\shubh\AppData\Local\Temp\ipykernel_976\935310350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\shubh\AppData\Local\Temp\ipykernel_976\935310350.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month
C:\Users\shubh\AppData\Local\Temp\ipykernel_976\935310350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [179]:
dimDate.head()

,fips,date,year,month,day_of_week
0,49.0,2021-02-20,2021,2,5
1,51.0,2021-02-20,2021,2,5
2,78.0,2021-02-20,2021,2,5
3,50.0,2021-02-20,2021,2,5
4,53.0,2021-02-20,2021,2,5


In [143]:
bucket = 'dexter-covid-project'

In [183]:
csv_buffer = StringIO()

In [184]:
csv_buffer

In [153]:
factCovid.to_csv(csv_buffer)

In [154]:
s3_resource = boto3.resource('s3',
                region_name="ap-south-1",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket, 'output/factCovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'KMKXCN6D36GMCH1N',
  'HostId': '2gy6ShiHx383o+t7ASRKfkfLqTqi2YXQtyNxG5bQkMTFPBHbvkoYwKHa8fRmR6n3/DRndMbFhXo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2gy6ShiHx383o+t7ASRKfkfLqTqi2YXQtyNxG5bQkMTFPBHbvkoYwKHa8fRmR6n3/DRndMbFhXo=',
   'x-amz-request-id': 'KMKXCN6D36GMCH1N',
   'date': 'Thu, 05 Jan 2023 13:08:15 GMT',
   'etag': '"2551539c527f78ee6e72d268a389300a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2551539c527f78ee6e72d268a389300a"'}

In [185]:
dimDate.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                region_name="ap-south-1",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket, 'output/dimDate.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'RJR18SKDQKM2B7SN',
  'HostId': 'hkfyblgKjbTlaS6TkQUakhFmfVzOF/s/kMrZIGEs3CXf//EbXpqI9eDwao0pxCHqZVweI6R4SIa42o98VZs3dQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hkfyblgKjbTlaS6TkQUakhFmfVzOF/s/kMrZIGEs3CXf//EbXpqI9eDwao0pxCHqZVweI6R4SIa42o98VZs3dQ==',
   'x-amz-request-id': 'RJR18SKDQKM2B7SN',
   'date': 'Thu, 05 Jan 2023 14:02:37 GMT',
   'etag': '"0e0ad93ebf5afcbc45850b27340ad485"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0e0ad93ebf5afcbc45850b27340ad485"'}

In [186]:
csv_buffer = StringIO()
dimHospital.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                region_name="ap-south-1",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket, 'output/dimHospital.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'KAFMCWJYE4VTPE30',
  'HostId': 'i0iGmrgIX8qUjiAXJS49VUcgrXZz0K93ySHo0TebR2CarsbpC1ae+2Rq4rqCFr4dazx0ZaohdZY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'i0iGmrgIX8qUjiAXJS49VUcgrXZz0K93ySHo0TebR2CarsbpC1ae+2Rq4rqCFr4dazx0ZaohdZY=',
   'x-amz-request-id': 'KAFMCWJYE4VTPE30',
   'date': 'Thu, 05 Jan 2023 14:03:52 GMT',
   'etag': '"134c76d76c4864e70d5675387bf98737"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"134c76d76c4864e70d5675387bf98737"'}

In [187]:
csv_buffer = StringIO()
dimRegion.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                region_name="ap-south-1",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket, 'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'A9EBPK0DKV5WJ2B7',
  'HostId': 'klWqIEsO019qTgxRVF9jZ24Yw5QgydOPPW+TmNL2+v6a3pCveWmtIPiHXuVA5bEi5e2Yucj4FKpG6Sjmq7yY7w==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'klWqIEsO019qTgxRVF9jZ24Yw5QgydOPPW+TmNL2+v6a3pCveWmtIPiHXuVA5bEi5e2Yucj4FKpG6Sjmq7yY7w==',
   'x-amz-request-id': 'A9EBPK0DKV5WJ2B7',
   'date': 'Thu, 05 Jan 2023 14:04:34 GMT',
   'etag': '"f6fa91f001e22cd45b773bed29d7c55b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f6fa91f001e22cd45b773bed29d7c55b"'}

In [159]:
dimDate_sql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
print(dimDate_sql)

dimRegion_sql = pd.io.sql.get_schema(dimRegion.reset_index(), 'dimRegion')
print(dimRegion_sql)

dimHospital_sql = pd.io.sql.get_schema(dimHospital.reset_index(), 'dimHospital')
print(dimHospital_sql)

factCovid_sql = pd.io.sql.get_schema(factCovid.reset_index(), 'factCovid')
print(factCovid_sql)

CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [161]:
pip install redshift_connector

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 112.1/112.1 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 4.7 MB/s eta 0:00:00
     -------------------------------------- 105.0/105.0 kB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [162]:
import redshift_connector

In [166]:
conn = redshift_connector.connect(
    host='redshift-cluster-1.cqd5znshhjvl.ap-south-1.redshift.amazonaws.com',
    database='dev',
    user='awsuser',
    password='Password1'
)

In [167]:
conn.autocommit = True

In [168]:
cursor = redshift_connector.Cursor = conn.cursor()

In [169]:
cursor.execute("""
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
""")

cursor.execute("""
CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

cursor.execute("""
CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")

cursor.execute("""
CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
""")

In [188]:
cursor.execute("""
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
""")

In [189]:
cursor.execute("""
copy dimDate from 's3://dexter-covid-project/output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::136178993549:role/redshift-s3'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

In [ ]:
cursor.execute("""
copy dimHospital from 's3://dexter-covid-project/output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::136178993549:role/redshift-s3'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

cursor.execute("""
copy dimRegion from 's3://dexter-covid-project/output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::136178993549:role/redshift-s3'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

cursor.execute("""
copy factCovid from 's3://dexter-covid-project/output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::136178993549:role/redshift-s3'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")